In [10]:
import requests
from bs4 import BeautifulSoup  # Assuming BeautifulSoup for HTML parsing

def scrape_website(url):
    response = requests.get(url)
    content = response.content

    # Parse HTML content based on website structure
    soup = BeautifulSoup(content, 'html.parser')

    # Extract relevant text using appropriate selectors (e.g., paragraphs, articles)
    text_content = soup.find_all('p')  # Example selector, adjust as needed

    return text_content


In [13]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

def store_content(content):
    # Connect to MILVUS server (replace with your connection details)
    connections.connect("default", host='localhost', port='19530')

    # Create a collection for storing content (adjust schema as needed)
    collection_name = 'website_content'

    # Define the schema
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="text", dtype=DataType.STRING)
    ]
    schema = CollectionSchema(fields, "Collection for storing website content")
    collection = Collection(name=collection_name, schema=schema)

    # Convert content to appropriate format for MILVUS (e.g., list of lists)
    documents = [[item] for item in content]

    # Insert documents into the collection
    collection.insert(documents)

    # Optionally, you can flush to ensure data is persisted
    collection.flush()

    # Disconnect from Milvus
    connections.disconnect("default")


In [14]:
cont = scrape_website('https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/')
store_content(cont)

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

In [1]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, MilvusException

def store_content(content):
    try:
        # Connect to MILVUS server (replace with your connection details)
        connections.connect("default", host='localhost', port='19530', timeout=10.0)

        # Create a collection for storing content (adjust schema as needed)
        collection_name = 'website_content'

        # Define the schema
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name="text", dtype=DataType.STRING)
        ]
        schema = CollectionSchema(fields, "Collection for storing website content")
        collection = Collection(name=collection_name, schema=schema)

        # Convert content to appropriate format for MILVUS (e.g., list of lists)
        documents = [[item] for item in content]

        # Insert documents into the collection
        collection.insert(documents)

        # Optionally, you can flush to ensure data is persisted
        collection.flush()

        # Disconnect from Milvus
        connections.disconnect("default")

    except MilvusException as e:
        print(f"An error occurred: {e}")

# Example usage
content = ["Document 1", "Document 2", "Document 3"]
store_content(content)


An error occurred: <SchemaNotReadyException: (code=1, message=No vector field is found.)>


In [9]:
import requests
from bs4 import BeautifulSoup
import sqlite3

def scrape_website(url):
    response = requests.get(url)
    content = response.content

    # Parse HTML content based on website structure
    soup = BeautifulSoup(content, 'html.parser')

    # Extract relevant text using appropriate selectors (e.g., paragraphs, articles)
    text_content = soup.find_all('p')  # Example selector, adjust as needed
    text_list = [p.get_text() for p in text_content]

    return text_list

def save_to_db(text_list, db_name="scraped_content.db"):
    # Connect to SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS content (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        paragraph TEXT NOT NULL
    )
    ''')

    # Insert text content into the table
    for paragraph in text_list:
        cursor.execute('''
        INSERT INTO content (paragraph) VALUES (?)
        ''', (paragraph,))

    # Commit and close the connection
    conn.commit()
    conn.close()

if __name__ == "__main__":
    url = "https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/"  # Replace with the target URL
    text_list = scrape_website(url)
    save_to_db(text_list)


In [5]:
scrape_website('https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/')

[<p class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/healthcare-ai-chatbot-using-llama-llm-langchain/" rel="bookmark" title="Healthcare AI ChatBot using LLAMA, LLM, Langchain">Healthcare AI ChatBot using LLAMA, LLM, Langchain</a></p>,
 <p class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/efficient-supply-chain-assessment-overcoming-technical-hurdles-for-web-application-development/" rel="bookmark" title="Efficient Supply Chain Assessment: Overcoming Technical Hurdles for Web Application Development">Efficient Supply Chain Assessment: Overcoming Technical Hurdles for Web Application Development</a></p>,
 <p class="entry-title td-module-title"><a href="https://insights.blackcoffer.com/streamlined-integration-interactive-brokers-api-with-python-for-desktop-trading-application/" rel="bookmark" title="Streamlined Integration: Interactive Brokers API with Python for Desktop Trading Application">Streamlined Integration: Interactive Brok

In [11]:
from pymilvus import MilvusClient

client = MilvusClient("scraped_content.db")

Failed to create new connection using: c4fc8ef34f294e938c0a0ff0eb84b637


ModuleNotFoundError: No module named 'milvus_lite'

In [3]:
from milvus import Milvus  # Assuming MILVUS for document storage

def store_content(content):
    # Connect to MILVUS server (replace with your connection details)
    milvus = Milvus(host='localhost', port=19530)

    # Create a collection for storing content (adjust schema as needed)
    collection_name = 'website_content'
    schema = {'schema': [
        {'field_name': 'text', 'field_type': 'TEXT', 'is_primary': True}
    ]}
    milvus.create_collection(collection_name, schema)

    # Convert content to appropriate format for MILVUS (e.g., list of dictionaries)
    documents = [{'text': item} for item in content]

    # Insert documents into the collection
    milvus.insert(collection_name, documents)

    milvus.disconnect()  # Close connection


ImportError: cannot import name 'Milvus' from 'milvus' (d:\New folder (2)\.venv\Lib\site-packages\milvus\__init__.py)